In [292]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import os

In [293]:
ndcs = pd.read_csv("../Data/20240118_countries.csv")
ndcs.head(5)

,ISO,Country,NDC,Date,Climate.Watch.HTML.File
0,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html
1,AGO,Angola,1,11/16/20,AGO-first_ndc-EN.html
2,AGO,Angola,2,5/31/21,AGO-revised_first_ndc-EN.html
3,ALB,Albania,1,9/21/16,ALB-first_ndc-EN.html
4,ALB,Albania,2,10/12/21,ALB-revised_first_ndc-EN.html


In [294]:
def extract_ndc(index):
    path_name = "../Data/20240117_ClimateWatch_AllData/NDC_text_HTML/ndc-master" + "/"

In [295]:
pd.read_csv("../Output/20240118_ndc_lines.csv")

,iso,country,ndc,date,html,elem,h1,h2,h3,h4,p,ol,ul,tab,hdr1,hdr2,hdr3,hdr4,line
0,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,10,2,1,0,0,1,0,0,0,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,NaN,21 September 2015
1,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,11,2,1,0,0,2,0,0,0,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,NaN,The Islamic Republic of Afghanistan hereby com...
2,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,13,2,1,0,0,0,0,0,1,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,NaN,Executive Summary Base Year: 2005 Target Years...
3,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,42,2,1,0,0,0,0,1,0,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,NaN,Adaptation: USD 10.785 billion\nMitigation: US...
4,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,45,2,1,0,0,3,0,0,0,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,NaN,\nFigure 1. Greenhouse Gas Emissions for Afgha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111147,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,1749,2,10,4,7,778,0,0,0,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,Tracking climate finance in Zimbabwe,"11th Floor, Kaguvi Building,"
111148,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,1750,2,10,4,7,779,0,0,0,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,Tracking climate finance in Zimbabwe,"Cnr S.V Muzenda Street/ Central Avenue, Harare..."
111149,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,1751,2,10,4,7,780,0,0,0,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,Tracking climate finance in Zimbabwe,Tel: +263 (242) 701681/3
111150,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,1752,2,10,4,7,781,0,0,0,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,Tracking climate finance in Zimbabwe,Email: climatechange@environment.gov.zw Websit...


In [296]:
def extract_ndc(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        
    # Extracting elements
    elements = soup.find_all()
    h1 = soup.find_all('h1')
    h2 = soup.find_all('h2')
    h3 = soup.find_all('h3')
    h4 = soup.find_all('h4')
    p = soup.find_all('p')
    ul = soup.find_all('ul')
    ol = soup.find_all('ol')
    tab = soup.find_all('table')
    
    # Creating a DataFrame to store the information
    struct = pd.DataFrame({
        'elem': range(len(elements)),
        'h1': [0]*len(elements),
        'h2': [0]*len(elements),
        'h3': [0]*len(elements),
        'h4': [0]*len(elements),
        'p': [0]*len(elements),
        'ol': [0]*len(elements),
        'ul': [0]*len(elements),
        'tab': [0]*len(elements)
    })
    
    # Marking the presence of tags
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in h1]), 'h1'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in h2]), 'h2'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in h3]), 'h3'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in h4]), 'h4'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in p]), 'p'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in ul]), 'ul'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in ol]), 'ol'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in tab]), 'tab'] = 1
    
    # Cumulative sums for tags
    struct['h1'] = struct['h1'].cumsum()
    struct['h2'] = struct['h2'].cumsum()
    struct['h3'] = struct['h3'].cumsum()
    struct['h4'] = struct['h4'].cumsum()
    struct['p'] = struct['p'].cumsum() * struct['p']
    struct['ol'] = struct['ol'].cumsum() * struct['ol']
    struct['ul'] = struct['ul'].cumsum() * struct['ul']
    struct['tab'] = struct['tab'].cumsum() * struct['tab']
    
    # Filtering rows with at least one of the specified tags
    struct2 = struct[struct[['p', 'ol', 'ul', 'tab']].sum(axis=1) > 0]

    element_indexed = [(i, e) for i, e in enumerate(elements)]
    tag_dict = {}
    for i, e in element_indexed:
        soup = BeautifulSoup(str(e), 'html.parser')
        tags = soup.find_all()
        if len(tags) > 0:
            parent_tag = tags[0].name
            if tag_dict.get(parent_tag):
                tag_dict[parent_tag].add(i)
            else:
                tag_dict[parent_tag] = set([i])
    
    valid_indice = list(struct2.index)
    struct2['line'] = [e[1].get_text() for e in element_indexed if e[0] in valid_indice]
    # Adding header texts
    # ndc_lines$hdr1[ndc_lines$h1 > 0] = hdr1[ndc_lines$h1[ndc_lines$h1 > 0]]

    # hdr1 = [BeautifulSoup(str(html), 'html.parser').get_text() for html in h1 if html is not None]
    # hdr1 = pd.Series(hdr1)
    # struct2.loc[struct2['h1'] > 0, 'hdr1'] = hdr1[struct2.loc[struct2['h1'] > 0, 'h1']]
    # print(struct2)

    # struct2['hdr1'] = h1_col
    # struct2['hdr2'] = h2_col
    # struct2['hdr3'] = h3_col
    # struct2['hdr4'] = h4_col
    # if e in struct2['elem'].values
    # Adding additional information from the ndcs DataFrame
    struct2['iso'] = ndcs['ISO'].iloc[0]
    struct2['country'] = ndcs['Country'].iloc[0]
    struct2['ndc'] = ndcs['NDC'].iloc[0]
    struct2['date'] = ndcs['Date'].iloc[0]
    struct2['html'] = file_path
    
    return struct2

In [297]:
ndc_lines = extract_ndc("../Data/20240117_ClimateWatch_AllData/NDC_text_HTML/ndc-master/AFG-first_ndc-EN.html")
ndc_lines
# # Filtering out empty lines
# ndc_lines = ndc_lines[ndc_lines['line'] != '']

C:\Users\User\AppData\Local\Temp\ipykernel_11236\3201950728.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  struct2['line'] = [e[1].get_text() for e in element_indexed if e[0] in valid_indice]
C:\Users\User\AppData\Local\Temp\ipykernel_11236\3201950728.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  struct2['iso'] = ndcs['ISO'].iloc[0]
C:\Users\User\AppData\Local\Temp\ipykernel_11236\3201950728.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

,elem,h1,h2,h3,h4,p,ol,ul,tab,line,iso,country,ndc,date,html
10,10,2,1,0,0,1,0,0,0,21 September 2015,AFG,Afghanistan,1,11/23/16,../Data/20240117_ClimateWatch_AllData/NDC_text...
11,11,2,1,0,0,2,0,0,0,The Islamic Republic of Afghanistan hereby com...,AFG,Afghanistan,1,11/23/16,../Data/20240117_ClimateWatch_AllData/NDC_text...
13,13,2,1,0,0,0,0,0,1,\n\n\nExecutive Summary\n\n\n\n\nBase Year:\n2...,AFG,Afghanistan,1,11/23/16,../Data/20240117_ClimateWatch_AllData/NDC_text...
42,42,2,1,0,0,0,0,1,0,Adaptation: USD 10.785 billion\nMitigation: U...,AFG,Afghanistan,1,11/23/16,../Data/20240117_ClimateWatch_AllData/NDC_text...
45,45,2,1,0,0,3,0,0,0,\nFigure 1. Greenhouse Gas Emissions for Afgha...,AFG,Afghanistan,1,11/23/16,../Data/20240117_ClimateWatch_AllData/NDC_text...
50,50,2,1,1,0,4,0,0,0,Introduction,AFG,Afghanistan,1,11/23/16,../Data/20240117_ClimateWatch_AllData/NDC_text...
53,53,2,1,1,1,5,0,0,0,Afghanistan has extensive development and clim...,AFG,Afghanistan,1,11/23/16,../Data/20240117_ClimateWatch_AllData/NDC_text...
58,58,2,1,1,1,6,0,0,0,Afghanistan is highly prone to natural disaste...,AFG,Afghanistan,1,11/23/16,../Data/20240117_ClimateWatch_AllData/NDC_text...
61,61,2,1,1,1,7,0,0,0,"Despite these challenges, Afghanistan can rema...",AFG,Afghanistan,1,11/23/16,../Data/20240117_ClimateWatch_AllData/NDC_text...
62,62,2,1,1,1,8,0,0,0,"Appropriate support in the form of finance, ca...",AFG,Afghanistan,1,11/23/16,../Data/20240117_ClimateWatch_AllData/NDC_text...
